# 1. Logit

In [223]:
import numpy as np
import pandas as pd
import os

from tqdm import tqdm

import scipy.optimize as op


## Load Dataset

In [1]:
data_path = 'D:/OneDrive - 연세대학교 (Yonsei University)/Lectures/2023-2_통행자행태모형/03_Logit/Data_combined.xlsx'

In [9]:
df_c = pd.read_excel(data_path, sheet_name = 'Choice')
df_p = pd.read_excel(data_path, sheet_name = 'Personal')
#df_a = pd.read_excel(data_path, sheet_name = 'Attitude')

In [8]:
df_c.head(3), df_c.shape

(     P_ID  Sit  Cset  Alti  ATime  Time  Fuel  Fare  Choice
 0  207109    1     3     1      0    20  0.20   0.0       0
 1  207109    1     3     2      2    25  0.15   0.0       1
 2  207109    1     3     3      8    25  0.00   0.2       0,
 (3600, 9))

In [10]:
df_p.head(3), df_p.shape

(     P_ID  Gender  Age
 0  207109       0   43
 1  207238       1   37
 2  207263       1   73,
 (300, 3))

## 속성별 Raw data 추출

In [39]:
Z = ['Age']# 개인의 사회경제적 변수
X = ['ATime', 'Time', 'Fuel', 'Fare']# 대안 속성 변수
Y = ['Choice'] # 선택지

In [96]:
R_ZJ = np.array(df_p[Z]) #> (300, 1)
# 개인의 사회경제적 속성이 변수에 미치는 영향. age 컬럼만 불러와서 numpy 배열로 만든다

In [41]:
R_XJ = np.array(df_c[X]) #> (3600 x 4)

In [42]:
R_YJ = np.array(df_c[Y]) #> (3600 x 1)

In [43]:
NofCset = 3 # Choice set이 3개임을 지정해 준다

## 주요변수 지정

In [51]:
NJ = NofCset # Number of Alternative
NC = int(len(df_c)/NJ) # 선택지 개수
NN = len(df_p) # 응답자 수 
NT = int(NC/NN) # 1인당 몇 개 선택지를 갖는가?

NZJ = len(Z) + 1 # 상수를 포함한다
NX =  len(X) # X변수 길이

## 차원 확대

In [58]:
ZJ = np.zeros(shape = (NC, NJ, NZJ)) #> (1200, 3, 2)
XJ = np.zeros(shape = (NC, NJ, NX)) #> (1200, 3, 4)
YJ = np.zeros(shape = (NC, NJ)) #> (1200, 3)

## 차원 확장
* MATLAB에서 `ZJ(:,1:(end-1),z)=repmat(R_ZJ(:,z-1),[1,NJ-1])`;
    * 2D 배열 R_ZJ NJ-1의 (z-1)번째 열을 수평으로 복제하여 3D 배열 ZJ의 z번째 슬라이스 값을 설정
    *  즉, ZJ의 처음 두 차원은 동일하게 유지되고 z번째 슬라이스의 마지막 열은 1로 설정된다.

In [65]:
ZJ[:, :-1, 0] = 1

In [196]:
if NT == 1:
    for z in range(1, NZJ): #> 2번째 항목부터 NZJ 번째까지
        ZJ[:, :-1, z] = np.tile(R_ZJ[:, z-1][:, None], (1, NJ-1))
        
else: # 선택지가 2개 이상인 경우
    for z in range(1, NZJ):
        k = 0
        
        for n in range(0, NC, NT):
            ZJ[n:n+NT, :-1, z] = np.tile(R_ZJ[k, z-1], (NT, NJ-1))
            k += 1 #> 여기까지는 ZJ Shape가 변하지 않는다. 계속 (1200, 3, 2) 여야 한다.

In [210]:
k = 0

for n in range(NC): #> XJ : (1200, 3, 4) >> , YJ : (1200, 3) >> 
    XJ[n, :, :] = np.reshape(R_XJ[k:k+NJ, :], (1, NJ, NX))
    YJ[n, :] = np.reshape(R_YJ[k:k+NJ, 0], (1, NJ))
    
    k += NJ

## 파라미터 개수 불러오기

In [11]:
param_path = 'D:/OneDrive - 연세대학교 (Yonsei University)/Lectures/2023-2_통행자행태모형/03_Logit/Parameter_Set_Up_Logit.xlsx'

In [13]:
df_param = pd.read_excel(param_path, sheet_name = 'Sheet1')

In [14]:
df_param

,Category,Index1,Index2,Category.1,Index1.1,Index2.1,Initial parameter
0,Bzi,Constant,Private car,1,1,1,0
1,NaN,NaN,Shared-car,1,1,2,0
2,NaN,Age,Private car,1,2,1,0
3,NaN,NaN,Shared-car,1,2,2,0
4,Bxi,Access time,All,2,1,0,0
5,NaN,Travel time,All,2,2,0,0
6,NaN,Fuel cost,PC &SC,2,3,0,0
7,NaN,Fare,Public transportation,2,4,3,0


In [217]:
NP = len(df_param) # Param 개수
p_index = np.array(df_param[['Category.1', 'Index1.1', 'Index2.1']]) #> (8,3) 행렬

In [221]:
param = np.array(df_param[['Initial parameter']]) #> (8,) 행렬. 여기에 결과가 저장될 것이다

## Estimation Criterion

In [222]:
MAXITERS = 5000 # Maximum Iteration 
PARAMTOL = 0.0000001 #  Maximum change in Paramters
LLTOL = 0.00000001 # Maximum change in Log-likelihood value